In [ ]:
pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
import pandas as pd
import json
import time
import os
import re

In [4]:
path = 'C:/Users/MSII/Downloads/selenium/'
os.chdir(path)

In [5]:
url_page = ["https://thai.tourismthailand.org/Search-result/attraction?destination_id=101&sort_by=datetime_updated_desc&page=1&perpage=15&menu=attraction",
              "https://thai.tourismthailand.org/Search-result/attraction?destination_id=101&sort_by=datetime_updated_desc&page=2&perpage=15&menu=attraction",
              "https://thai.tourismthailand.org/Search-result/attraction?destination_id=101&sort_by=datetime_updated_desc&page=3&perpage=15&menu=attraction",
              "https://thai.tourismthailand.org/Search-result/attraction?destination_id=101&sort_by=datetime_updated_desc&page=4&perpage=15&menu=attraction"
]

In [6]:
def get_links_from_page(url):
    # Open webdriver
    driver = webdriver.Chrome(path+"chromedriver.exe")
    driver.get(url)
    driver.implicitly_wait(5)

    # Find links on the page
    links = driver.find_elements(By.XPATH, '//div[@class="wrap-default-cards-articles"]/a')
    page_links = [link.get_attribute('href') for link in links]

    # Close the webdriver
    driver.quit()

    return page_links

In [7]:
# List to store links from page
all_links = []

# Iterate through the page URLs and extract url
for url in url_page:
    links = get_links_from_page(url)
    all_links.extend(links)

all_links

C:\Users\MSII\AppData\Local\Temp\ipykernel_18280\2778052813.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path+"chromedriver.exe")


['https://thai.tourismthailand.org/Attraction/%E0%B8%A7%E0%B8%B1%E0%B8%94%E0%B8%9B%E0%B9%88%E0%B8%B2%E0%B8%95%E0%B8%B6%E0%B8%87',
 'https://thai.tourismthailand.org/Attraction/%E0%B8%A7%E0%B8%B1%E0%B8%94%E0%B8%94%E0%B8%AD%E0%B8%A2%E0%B9%81%E0%B8%A1%E0%B9%88%E0%B8%9B%E0%B8%B1%E0%B9%8B%E0%B8%87',
 'https://thai.tourismthailand.org/Attraction/%E0%B9%80%E0%B8%AA%E0%B8%B2%E0%B8%AD%E0%B8%B4%E0%B8%99%E0%B8%97%E0%B8%82%E0%B8%B4%E0%B8%A5-%E0%B9%80%E0%B8%AA%E0%B8%B2%E0%B8%AB%E0%B8%A5%E0%B8%B1%E0%B8%81%E0%B9%80%E0%B8%A1%E0%B8%B7%E0%B8%AD%E0%B8%87',
 'https://thai.tourismthailand.org/Attraction/%E0%B8%AD%E0%B8%B8%E0%B8%97%E0%B8%A2%E0%B8%B2%E0%B8%99%E0%B8%AB%E0%B8%A5%E0%B8%A7%E0%B8%87%E0%B8%A3%E0%B8%B2%E0%B8%8A%E0%B8%9E%E0%B8%A4%E0%B8%81%E0%B8%A9%E0%B9%8C',
 'https://thai.tourismthailand.org/Attraction/%E0%B8%AD%E0%B8%B8%E0%B8%97%E0%B8%A2%E0%B8%B2%E0%B8%99%E0%B9%81%E0%B8%AB%E0%B9%88%E0%B8%87%E0%B8%8A%E0%B8%B2%E0%B8%95%E0%B8%B4%E0%B9%81%E0%B8%A1%E0%B9%88%E0%B9%82%E0%B8%96',
 'https://thai.tourismtha

In [8]:
def extract_data(url):
    driver = webdriver.Chrome(path+"chromedriver.exe")
    driver.get(url)
    driver.implicitly_wait(5)
    
    try:
        actions = ActionChains(driver)
        cookies = driver.find_element(By.CSS_SELECTOR, 'button.btn.btn-primary.w-100')
        actions.click(cookies).perform()
        see_more = driver.find_element(By.CSS_SELECTOR, 'div.highlight-and-fact-see-more-btn')
        actions.move_to_element(see_more).click().perform()
        time.sleep(1)
        to_know = driver.find_element(By.ID, 'things_to_know')
        button = driver.find_element(By.CSS_SELECTOR, 'button.btn.btn-opening.d-block.font-weight-bold.font-family-suksa.mt-2.btn-secondary')
        actions.move_to_element(to_know).click(button).perform()
        time.sleep(1)
        tooltip = driver.find_element(By.CSS_SELECTOR, 'div.tooltip-inner').text

    except NoSuchElementException:
        tooltip = ""
        
    # Elements to extract with their CSS selectors and XPATH
    elements_to_extract = {
        "name": 'h3.headline.d-flex.w-auto',
        "type": 'p.business-type-title',
        "type_data": 'p.business-type-name.mb-3',
        "contact": 'div.d-block.business-contact',
        "contact_data": 'ul.list-unstyled.business-contact-info',
        "sec": '//*[@id="__layout"]/div/div[1]/div/div[3]/div/div[2]/div[1]/div/div/div[1]/div[2]/span/div',
        "sec_data": '//*[@id="__layout"]/div/div[1]/div/div[3]/div/div[2]/div[1]/div/div/div[2]/div/div/div/div',
        "facility": '//*[@id="__layout"]/div/div[1]/div/div[3]/div/div[2]/div[3]/div/div[1]/div[2]/span/ul/li/div',
        "facility_data": '//*[@id="__layout"]/div/div[1]/div/div[3]/div/div[2]/div[3]/div/div[2]/div/div/div/div',
        "gen": '//*[@id="interesting_facts"]/div/div[1]/div[2]/span/ul/li/div',
        "gen_data": '//*[@id="interesting_facts"]/div/div[2]/div/div/div',
        "impo": '//*[@id="__layout"]/div/div[1]/div/div[3]/div/div[2]/div[5]/div/div/div[1]/div[2]/span/div',
        "impo_data": '//*[@id="__layout"]/div/div[1]/div/div[3]/div/div[2]/div[5]/div/div/div[2]/div/div/div/div'
    }

    extracted_data = {}

    for key, selector in elements_to_extract.items():
        try:
            if selector.startswith('/'):
                # If the selector starts with '/', it is an XPath
                element = driver.find_element(By.XPATH, selector).text

            elif key == "contact":
                # Because CSS_SELECTOR = 'p.mb-0' is used repeatedly in multiple elements
                element = driver.find_element(By.CSS_SELECTOR, selector).find_element(By.CSS_SELECTOR, 'p.mb-0').text
            
            else:
                # Otherwise, assume it is a CSS selector
                element = driver.find_element(By.CSS_SELECTOR, selector).text

            # Check if the key is 'contact' and if "go to website" is present in the text
            if key == "contact_data" and 'go to website' in element:
                # Replace "go to website" with the actual website link
                website_link = driver.find_element(By.CSS_SELECTOR, 'a.mr-1').get_attribute('href')
                element = element.replace('go to website', website_link)

            # Check if 'sec_data' contains the word 'เวลาทำการ' and replace the word after with the tooltip value
            if key == "sec_data" and 'เวลาทำการ' in element:
                element = re.sub(r'เวลาทำการ\s*\S+', f'เวลาทำการ {tooltip}', element)

            # Remove newline characters from the text
            element = element.replace('\n', " ")
            element = element.replace('"', "")

            # Check if the element contains only special characters
            if re.match(r'^[^A-Za-z0-9ก-๙]+$', element):
                element = ""
                
        except NoSuchElementException:
            element = ""
        
        extracted_data[key] = element

    # Check conditions and update values accordingly
    for data, data_key in [('sec', 'sec_data'), ('facility', 'facility_data'), ('gen', 'gen_data'), ('impo', 'impo_data')]:
        if extracted_data[data_key] == "" or extracted_data[data_key] == " ":
            extracted_data[data] = ""
            extracted_data[data_key] = ""

    table_dict = {
        "name": f"{extracted_data['name']}",
        "type": f"{extracted_data['type']}{extracted_data['type_data']}",
        "contact": f"{extracted_data['contact']}{extracted_data['contact_data']}",
        "section": f"{extracted_data['sec']}{extracted_data['sec_data']}",
        "facility": f"{extracted_data['facility']}{extracted_data['facility_data']}",
        "general": f"{extracted_data['gen']}{extracted_data['gen_data']}",
        "important": f"{extracted_data['impo']}{extracted_data['impo_data']}"
    }

    # Create a dictionary with the extracted data
    content_dict = {
        'content': f"{extracted_data['name']} เป็น{extracted_data['type']}แบบ{extracted_data['type_data']} \
{extracted_data['contact']}{extracted_data['contact_data']} {extracted_data['sec_data']} \
{extracted_data['facility']}{extracted_data['facility_data']} \
{extracted_data['gen']}{extracted_data['gen_data']} \
{extracted_data['impo']}{extracted_data['impo_data']}"
    }

    driver.quit()
    return table_dict, content_dict

In [9]:
# List to store extracted data
table_list = []
content_list = []

# Iterate through the URLs and extract data
for link in range(0,len(all_links)):
    table, content = extract_data(all_links[link])
    table_list.append(table)
    content_list.append(content)

C:\Users\MSII\AppData\Local\Temp\ipykernel_13076\3076115809.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path+"chromedriver.exe")


In [10]:
table_df = pd.DataFrame(table_list)

# Save the DataFrame to an Excel file
table_df.to_excel('table_list.xlsx', index=False)

# Print the extracted data
print(table_df)

                                            name  \
0                                      วัดป่าตึง   
1                                  วัดดอยแม่ปั๋ง   
2                      เสาอินทขิล (เสาหลักเมือง)   
3                             อุทยานหลวงราชพฤกษ์   
4                            อุทยานแห่งชาติแม่โถ   
5                           อุทยานแห่งชาติแม่วาง   
6                          อุทยานแห่งชาติออบหลวง   
7                                  ถ้ำหลวงแม่สาบ   
8                                 หมู่บ้านแม่ลาย   
9                                  วัดหลวงขุนวิน   
10                     พระตำหนักภูพิงคราชนิเวศน์   
11                             สวนสัตว์เชียงใหม่   
12                    พิพิธภัณฑ์เรือนโบราณล้านนา   
13                          วัดเจดีย์หลวงวรวิหาร   
14  พระมหาธาตุนภเมทนีดลและพระมหาธาตุนภพลภูมิสิริ   
15                                      ม่อนแจ่ม   
16                         วัดพระสิงห์วรมหาวิหาร   
17                                  วัดเชียงมั่น   
18          

In [11]:
# Convert the list of dictionaries to JSON format
json_table = json.dumps(table_list, ensure_ascii=False, indent=2)

# Save the JSON data to a file
with open('json_table.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_table)

# Print the JSON data
print(json_table)

[
  {
    "name": "วัดป่าตึง",
    "type": "ประเภท สถานที่ท่องเที่ยววัด",
    "contact": "ข้อมูลติดต่อเบอร์โทร :+66 5388 0061",
    "section": "ข้อมูลที่อยู่ หมู่ 7 สันกำแพง, เชียงใหม่",
    "facility": "เครื่องอำนวยความสะดวกที่จอดรถ",
    "general": "ข้อมูลทั่วไปหลวงปู่หล้า ตาทิพย์ วัดป่าตึง ตั้งอยู่ในเขตตำบลออนใต้ อำเภอสันกำแพง เดิมเป็นวัดร้าง คู่กับวัดเชียงแสน ก่อนที่จะสร้างวัดนี้ขึ้นมาได้พบพระบรมสารีริกธาตุ และของมีค่ามากมายหลายอย่าง ความน่าสนใจของวัดป่าตึง คือการเป็นวัดเก่าคู่กับวัดเชียงแสน แม้จะเป็นวัดร้างก็ตาม แต่ก็มีการค้นพบพระบรมสารีริกธาตุและของมีค่ามากมายหลายอย่าง อาทิ วัตถุโบราณ พระพุทธรูป เครื่องถ้วยชามสังคโลก ซึ่งปัจจุบันเก็บอยู่ในพิพิธภัณฑ์เครื่องถ้วยชามที่ตั้งอยู่ภายในบริเวณวัด นอกจากนี้ภายในศาลาการเปรียญยังเป็นที่ตั้งศพของเกจิอาจารย์ชื่อดัง คือหลวงปู่หล้า ซึ่งสังขารของท่านไม่เน่าเปื่อยให้ผู้ที่มีศรัทธาได้บูชา ในขณะที่ท่านมีชีวิตอยู่นั้น ท่านได้รับสมญานามจากศรัทธาญาติโยมว่าเป็นพระภิกษุที่มีญาณวิเศษ ที่สามารถล่วงรู้เหตุการณ์ในอนาคตได้ หลายคนจึงขนานนามท่านว่า หลวงปู่หล้า 

In [13]:
# Convert the list of dictionaries to JSON format
json_content = json.dumps(content_list, ensure_ascii=False, indent=2)

# Save the JSON data to a file
with open('json_content.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_content)

# Print the JSON data
print(json_content)

[
  {
    "content": "วัดป่าตึง เป็นประเภท สถานที่ท่องเที่ยวแบบวัด ข้อมูลติดต่อเบอร์โทร :+66 5388 0061 ที่อยู่ หมู่ 7 สันกำแพง, เชียงใหม่ เครื่องอำนวยความสะดวกที่จอดรถ ข้อมูลทั่วไปหลวงปู่หล้า ตาทิพย์ วัดป่าตึง ตั้งอยู่ในเขตตำบลออนใต้ อำเภอสันกำแพง เดิมเป็นวัดร้าง คู่กับวัดเชียงแสน ก่อนที่จะสร้างวัดนี้ขึ้นมาได้พบพระบรมสารีริกธาตุ และของมีค่ามากมายหลายอย่าง ความน่าสนใจของวัดป่าตึง คือการเป็นวัดเก่าคู่กับวัดเชียงแสน แม้จะเป็นวัดร้างก็ตาม แต่ก็มีการค้นพบพระบรมสารีริกธาตุและของมีค่ามากมายหลายอย่าง อาทิ วัตถุโบราณ พระพุทธรูป เครื่องถ้วยชามสังคโลก ซึ่งปัจจุบันเก็บอยู่ในพิพิธภัณฑ์เครื่องถ้วยชามที่ตั้งอยู่ภายในบริเวณวัด นอกจากนี้ภายในศาลาการเปรียญยังเป็นที่ตั้งศพของเกจิอาจารย์ชื่อดัง คือหลวงปู่หล้า ซึ่งสังขารของท่านไม่เน่าเปื่อยให้ผู้ที่มีศรัทธาได้บูชา ในขณะที่ท่านมีชีวิตอยู่นั้น ท่านได้รับสมญานามจากศรัทธาญาติโยมว่าเป็นพระภิกษุที่มีญาณวิเศษ ที่สามารถล่วงรู้เหตุการณ์ในอนาคตได้ หลายคนจึงขนานนามท่านว่า หลวงปู่หล้า ตาทิพย์ ท่านเป็นพระเกจิอีกรูปหนึ่งที่ปฏิบัติดี ปฏิบัติชอบ ตั้งตนอยู่ในหลักศีลธรรมอัน

### This code is written after collecting data. And additional corrections were made to bring information to use in Fine-Tuning.

In [ ]:
# Extract and combine data from key="name", "type", "general" for each entry
assistant_content = [
    f"{entry['general']}"
    for entry in json_table
]

# Remove "ข้อมูลทั่วไป" from the beginning of each entry in assistant_content
assistant_content = [assis.replace("ข้อมูลทั่วไป", "") for assis in assistant_content]

# Print the resulting list
print(assistant_content)

In [ ]:
# Add the desired prefix to each element in result_list and store in user_content list
user_content = [f"ชื่อสถานที่: {entry['name']}\n\n{entry['type']}\n\nข้อมูลรายละเอียดสถานที่: "
                for entry in json_table]

# Print the resulting list
print(user_content)

In [ ]:
# Create system_content by adding the specified word to each element in user_content
system_content = ["You are a friendly and polite guide who give the travel place information in Thailand." for content in user_content]

# Print the resulting list
print(system_content)

In [ ]:
# Format the data into the desired format
data_role = [
    {"messages": [
        {"role": "system", "content": system_content[i]},
        {"role": "user", "content": user_content[i]},
        {"role": "assistant", "content": assistant_content[i]}
    ]}
    for i in range(len(system_content))
]

# Convert the list of dictionaries to JSON format with all values on the same line
data_role_json = "\n".join(json.dumps(entry, ensure_ascii=False, separators=(', ', ': ')) for entry in data_role)

# Save the formatted data to a JSON file
with open('data_role.jsonl', 'w', encoding='utf-8') as json_file:
    json_file.write(data_role_json)

In [ ]:
# Create a list of dictionaries
data_prompt = [
    {"prompt": user_content[i], "completion": assistant_content[i]}
    for i in range(len(user_content))
]

# Convert the list of dictionaries to JSON format with all values on the same line
data_prompt_json = "\n".join(json.dumps(entry, ensure_ascii=False, separators=(', ', ': ')) for entry in data_prompt)

# Save the formatted data to a JSON file
with open('data_prompt.jsonl', 'w', encoding='utf-8') as json_file:
    json_file.write(data_prompt_json)